# Block Encoding Interface

High level bloqs for defining bloq encodings and operations on block encodings.

In [1]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `BlockEncoding`
Abstract interface for an arbitrary block encoding.

In general, given an $s$-qubit operator $H$ then the $(s+a)$-qubit unitary $B[H] = U$ is
a $(\alpha, a, \epsilon)$-block encoding of $H$ if it satisfies:

$$
    \lVert H - \alpha (\langle G|_a\otimes I_s U |G\rangle_a \otimes I_s) \rVert
    \le \epsilon,
$$

where $a$ is an ancilla register and $s$ is the system register, $U$ is a unitary sometimes
called a signal oracle, $\alpha$ is a normalization constant chosen such
that  $\alpha \ge \lVert H\rVert$ (where $\lVert \cdot \rVert$ denotes the
spectral norm), and $\epsilon$ is the precision to which the block encoding
is prepared. The state $|G\rangle_a$ is sometimes called the signal state,
and its form depends on the details of the block encoding.

For LCU based block encodings with $H = \sum_l w_l U_l$
we have
$$
U = \sum_l |l\rangle\langle l| \otimes U_l
$$
and $|G\rangle = \sum_l \sqrt{\frac{w_l}{\alpha}}|l\rangle_a$, which define the
usual SELECT and PREPARE oracles.

Other ways of building block encodings exist so we define the abstract base
class `BlockEncoding` bloq, which expects values for $\alpha$, $\epsilon$,
system and ancilla registers and a bloq which prepares the state $|G\rangle$.

Users must specify:
1. the normalization constant $\alpha \ge \lVert A \rVert$, where
    $\lVert \cdot \rVert$ denotes the spectral norm.
2. the precision to which the block encoding is to be prepared ($\epsilon$).

Developers must provide a method to return a bloq to prepare $|G\rangle$.

#### References
 - [Hamiltonian Simulation by Qubitization](https://quantum-journal.org/papers/q-2019-07-12-163/).     Low et al. 2019. Sec 2 and 3 for introduction and definition of terms.
 - [The power of block-encoded matrix powers: improved regression techniques via faster Hamiltonian simulation](https://arxiv.org/abs/1804.01973).     Chakraborty et al. 2018. Definition 3 page 8.


In [2]:
from qualtran.bloqs.block_encoding import BlockEncoding